# INTRODUCTION

In [ ]:
# # Clean up folders (metastore and spark-warehouse)
# import os
# import shutil

# for item in ["metastore_db", "spark-warehouse"]:
#     if os.path.exists(item):
#         shutil.rmtree(item)

In [ ]:

# Import SparkSession
import pyspark
from delta import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("STREAMING_DWH") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()


In [ ]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
df = spark \
    .readStream \
    .format("json") \
    .load("data/product/")

df.printSchema()

In [ ]:
from pyspark.sql.functions import current_timestamp, input_file_name

def create_bronze_streaming_table(source, target):

    # Generates a source path based on table name, reads all files from that and inserts into bronze schema
    query = (
        spark.readStream
        .format("json")
        .load(source)
        .withColumn("meta_ingestion_ts", current_timestamp())
        .withColumn("meta_filename", input_file_name())
        .writeStream
        .outputMode("append")
        .format("delta")
        .option("checkpointLocation", f"spark-warehouse/_checkpoints/{target}")
        .toTable(target)
    )
    return query

query1 = create_bronze_streaming_table(source="data/inventory", target="bronze_inventory")
query2 = create_bronze_streaming_table(source="data/product", target="bronze_product")
query3 = create_bronze_streaming_table(source="data/purchase", target="bronze_purchase")

# Use the code 
# spark.streams.awaitAnyTermination()


# CREATE SILVER TABLES
This function will create empty tables for future use.

Depending on the SCD type, the following columns will be created:

SCD type 1:
- meta_hashdiff
- meta_last_updated
- meta_sequence

SCD type 2:
- meta_hashdiff
- meta_is_current
- meta_valid_from
- meta_valid_to
- meta_sequence


In [ ]:
def create_silver_table(
        table_name : str, 
        surrogate_key : str, 
        source_table : str, 
        scd_type : str
    ):

    # Define table name and surrogate key
    query = f"CREATE TABLE IF NOT EXISTS {table_name} ({surrogate_key} string,"
    
    # Get schema of source table
    source_schema = spark.sql(f"describe table {source_table}").collect()
    for row in source_schema:
        query += f" {row['col_name']} {row['data_type']},"

    # Add extra meta columns depending on SCD (slowly changing dimension) type
    if scd_type == 1:
        query += "meta_hashdiff string, meta_last_updated timestamp, meta_sequence int) USING DELTA;"
    elif scd_type == 2:
        query += "meta_hashdiff string, meta_is_current boolean, meta_valid_from timestamp, meta_valid_to timestamp, meta_sequence int) USING DELTA;"

    print(query)
    spark.sql(query)

create_silver_table(table_name="silver_purchase_scd1", surrogate_key="transaction_sid", source_table="bronze_purchase", scd_type=1)
create_silver_table(table_name="silver_inventory_scd1", surrogate_key="inventory_sid", source_table="bronze_inventory", scd_type=1)
create_silver_table(table_name="silver_product_scd2", surrogate_key="product_sid", source_table="bronze_product", scd_type=2)


In [ ]:
spark.sql(f"select * from silver_product_scd2").show(5)
# spark.sql(f"select * from silver_purchase_scd1").show(5)

# SILVER TABLES: SLOWLY CHANGING DIMENSIONS (SCD) - TYPE 1 & 2

In [ ]:
from pyspark.sql.functions import md5, concat_ws, lit, row_number
from pyspark.sql.types import BooleanType, TimestampType
from pyspark.sql.window import Window

def create_silver_scd1_table(
    source : str, 
    target : str,
    timestamp_key : str,
    merge_key: str,
    surrogate_key : str,
    delta_load_column: str
):
    
    # Perform delta load
    df = spark.sql(f"select * from {source} where {delta_load_column} > (select coalesce(max({delta_load_column}), '1970-01-01') from {target})")

    # Calculate surrogate key as hash of natural key columns
    df = df.withColumn(surrogate_key, md5(merge_key))

    # Calculate hashdiff string based on all columns that doesn't contain "meta_" in the name
    df = df.withColumn("meta_hashdiff", md5(concat_ws("||", *[c for c in df.columns if "meta_" not in c])))

    # Set default values for meta_last_updated
    df = df.withColumn("meta_last_updated", current_timestamp())

    # Calculate sequence numbers if source data contain multiple rows for each merge_key
    window_spec = Window.partitionBy(merge_key).orderBy(timestamp_key)
    df = df.withColumn("meta_sequence", row_number().over(window_spec))
    
    # Create view with source data
    tmp_view_name = "temporaryView"
    df.createOrReplaceTempView(tmp_view_name)

    # Get list of sequences
    lst_sequence = sorted([p.meta_sequence for p in df.select('meta_sequence').distinct().collect()])

    # Run SCD1 table
    for seq_num in lst_sequence:
        print(f"Inserting into SILVER SCD TYPE 1 TABLE: {target}")
        merge_query = f"""
            MERGE INTO {target} AS target
            USING (
                SELECT * 
                FROM {tmp_view_name}
                WHERE meta_sequence = {seq_num}
            ) AS source ON target.{surrogate_key} = source.{surrogate_key}
            WHEN MATCHED AND target.meta_hashdiff <> source.meta_hashdiff 
                THEN UPDATE SET *
            WHEN NOT MATCHED 
                THEN INSERT *
        """
        spark.sql(merge_query).show()


In [ ]:

create_silver_scd1_table (
    source="bronze_purchase",
    target="silver_purchase_scd1",
    timestamp_key="transaction_time",
    merge_key="transaction_id",
    surrogate_key="transaction_sid",
    delta_load_column="transaction_time"
)

create_silver_scd1_table (
    source="bronze_inventory",
    target="silver_inventory_scd1",
    timestamp_key="event_time",
    merge_key="event_time",
    surrogate_key="inventory_sid",
    delta_load_column="event_time"
)

In [ ]:
def create_silver_scd2_table(
    source: str, 
    target: str, 
    merge_key: str, 
    timestamp_key: str, 
    surrogate_key : str,
    delta_load_column: str
):
    
    # Perform delta load
    df = spark.sql(f"select * from {source} where {delta_load_column} > (select coalesce(max({delta_load_column}), '1970-01-01') from {target})")

    # Calculate surrogate key as hash of natural key columns
    df = df.withColumn(surrogate_key, md5(merge_key))

    # Calculate hashdiff string based on all columns that doesn't contain "meta_" in the name
    df = df.withColumn("meta_hashdiff", md5(concat_ws("||", *[c for c in df.columns if "meta_" not in c])))

    # Set default values for meta columns
    df = df.withColumn("meta_is_current", lit(1).cast(BooleanType()))
    df = df.withColumn("meta_valid_from", df[timestamp_key])
    df = df.withColumn("meta_valid_to", lit('9999-12-31').cast(TimestampType()))

    # Calculate sequence numbers if source data contain multiple rows for each merge_key
    window_spec = Window.partitionBy(merge_key).orderBy(timestamp_key)
    df = df.withColumn("meta_sequence", row_number().over(window_spec))

    # Create an empty Delta table with the same schema
    tmp_view_name = "temporaryView"
    df.createOrReplaceTempView(tmp_view_name)

    # Get list of sequences
    lst_sequence = sorted([p.meta_sequence for p in df.select('meta_sequence').distinct().collect()])

    # Run SCD2 table 
    for seq_num in lst_sequence:
        print(f"Inserting into SILVER SCD TYPE 2 TABLE: {target}")
        merge_query = f"""
            MERGE INTO {target} AS target
            USING (
                SELECT * 
                FROM {tmp_view_name}
                WHERE meta_sequence = {seq_num}
            ) AS source ON target.{merge_key} = source.{merge_key}
            WHEN MATCHED AND target.meta_is_current = true AND target.meta_hashdiff <> source.meta_hashdiff
                THEN UPDATE SET meta_is_current = false, meta_valid_to = source.{timestamp_key}
            WHEN NOT MATCHED 
                THEN INSERT *
        """
        spark.sql(merge_query).show()

        insert_query = f"""
            INSERT INTO {target}
            SELECT * FROM 
            (
                SELECT source.*
                FROM {tmp_view_name} source
                JOIN {target} target ON target.{merge_key} = source.{merge_key}
                WHERE source.meta_sequence = {seq_num}
                AND target.meta_hashdiff <> source.meta_hashdiff 
            )
        """
        spark.sql(insert_query)

In [ ]:
# Create SCD2 tables
create_silver_scd2_table(
    source = "bronze_product",
    target = "silver_product_scd2",
    merge_key = "product_id",
    timestamp_key = "event_time",
    surrogate_key = "product_sid",
    delta_load_column="event_time"
)

In [ ]:
spark.sql("select * from silver_purchase_scd1").show(5)
spark.sql("select * from silver_inventory_scd1").show(5)
spark.sql("select * from silver_product_scd2 order by product_sid").show(5)

# CREATE GOLD TABLES - FACTS AND DIMENSIONS

In [ ]:

def create_gold_table(
        table_name : str, 
        surrogate_key : str, 
        source_table : str, 
        dim_table_refs : dict
    ):

    # Define table name and surrogate key
    query = f"CREATE TABLE IF NOT EXISTS {table_name} ({surrogate_key} string"

    # Loop through and add surrogate keys for foreign keys
    for row in dim_table_refs:
        query += f", {row['surrogate_key']} string"

    # Get schema of source table
    source_schema = spark.sql(f"describe table {source_table}").collect()
    for row in source_schema:
        if row['col_name'] != surrogate_key:
            query += f", {row['col_name']} {row['data_type']}"

    query += ") USING DELTA;"

    print(query)
    spark.sql(query)
    return


create_gold_table (
    table_name="gold_fact_purchase",
    source_table="silver_purchase_scd1",
    surrogate_key="transaction_sid",
    dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}]
)

create_gold_table (
    table_name="gold_fact_inventory",
    source_table="silver_inventory_scd1",
    surrogate_key="inventory_sid",
    dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}]
)

In [ ]:
from utils import generate_dim_table_references

def create_gold_fact_table(
    source : str, 
    target : str,
    surrogate_key : str,
    timestamp_key : str,
    dim_table_refs : dict,
    delta_load_column: str
):

    # Generate and run SQL query
    df = spark.sql(generate_dim_table_references(source=source,
                                                 target=target,
                                                 timestamp_key=timestamp_key, 
                                                 dim_table_refs=dim_table_refs, 
                                                 delta_load_column=delta_load_column))

    # Create an empty Delta table with the same schema
    tmp_view_name = "temporaryView"
    df.createOrReplaceTempView(tmp_view_name)

    # Merge into target table 
    merge_query = f"""
        MERGE INTO {target} AS target
        USING {tmp_view_name} AS source ON target.{surrogate_key} = source.{surrogate_key}
        WHEN MATCHED AND target.meta_hashdiff <> source.meta_hashdiff THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """
    spark.sql(merge_query).show()
    

In [ ]:
create_gold_fact_table (
    source="silver_purchase_scd1",
    target="gold_fact_purchase",
    surrogate_key="transaction_sid",
    timestamp_key="transaction_time",
    dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}],
    delta_load_column="transaction_time"
)

create_gold_fact_table (
    source="silver_inventory_scd1",
    target="gold_fact_inventory",
    surrogate_key="inventory_sid",
    timestamp_key="event_time",
    dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}],
    delta_load_column="event_time"
)


In [ ]:
# spark.sql("select * from gold_fact_purchase").show(5)
spark.sql("select * from gold_fact_inventory").show(5)

# PUTTING IT ALL TOGETHER

In [ ]:



while True:

    # BRONZE
    query1 = create_bronze_streaming_table(source="data/inventory", target="bronze_inventory")
    query2 = create_bronze_streaming_table(source="data/product", target="bronze_product")
    query3 = create_bronze_streaming_table(source="data/purchase", target="bronze_purchase")

    # SILVER
    create_silver_scd1_table (
        source="bronze_purchase",
        target="silver_purchase_scd1",
        timestamp_key="transaction_time",
        merge_key="transaction_id",
        surrogate_key="transaction_sid",
        delta_load_column="transaction_time"
    )
    create_silver_scd1_table (
        source="bronze_inventory",
        target="silver_inventory_scd1",
        timestamp_key="event_time",
        merge_key="event_time",
        surrogate_key="inventory_sid",
        delta_load_column="event_time"
    )
    create_silver_scd2_table(
        source = "bronze_product",
        target = "silver_product_scd2",
        merge_key = "product_id",
        timestamp_key = "event_time",
        surrogate_key = "product_sid",
        delta_load_column="event_time"
    )

    # GOLD
    create_gold_fact_table (
        source="silver_purchase_scd1",
        target="gold_fact_purchase",
        surrogate_key="transaction_sid",
        timestamp_key="transaction_time",
        dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}],
        delta_load_column="transaction_time"
    )

    create_gold_fact_table (
        source="silver_inventory_scd1",
        target="gold_fact_inventory",
        surrogate_key="inventory_sid",
        timestamp_key="event_time",
        dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}],
        delta_load_column="event_time"
    )

In [ ]:
spark.sql("select * from silver_product_scd2 order by product_sid").show(15)